In [7]:
import sys
print(sys.executable)

/home/mkrishne/default/bin/python3


In [2]:
!pwd

/home/mkrishne/PL_competition


In [8]:
!{sys.executable} -m pip install torch_geometric torch_scatter torch_sparse torch_cluster pyg_lib

  Using cached torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement pyg_lib (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for pyg_lib


In [69]:
import numpy as np
import os
from sklearn.metrics import pairwise_distances

# --- Configuration ---
base_dir = "/home/mkrishne/PL_competition/extracted_regions/train"
folder_names = [
    "tx_sphere_Tx_1", "tx_sphere_Tx_10", "tx_sphere_Tx_11", "tx_sphere_Tx_13",
    "tx_sphere_Tx_15", "tx_sphere_Tx_16", "tx_sphere_Tx_17", "tx_sphere_Tx_18",
    "tx_sphere_Tx_19", "tx_sphere_Tx_2", "tx_sphere_Tx_3", "tx_sphere_Tx_4",
    "tx_sphere_Tx_6", "tx_sphere_Tx_7", "tx_sphere_Tx_8"
]
npz_filename = "pair00000_tx_sphere50.npz"
key_name = "Tx"
num_points_to_select = 3
# ---------------------

print(f"--- Reading Tx coordinates and selecting {num_points_to_select} diverse points ---")

all_coords_list = []
all_folder_names = []

# --- 1. Load all coordinates ---
for folder in folder_names:
    file_path = os.path.join(base_dir, folder, npz_filename)
    try:
        data = np.load(file_path)
        if key_name in data:
            tx_coords = data[key_name]
            # Ensure coordinates are 1D array of 3 numbers
            if tx_coords.ndim == 1 and tx_coords.shape[0] == 3:
                 all_coords_list.append(tx_coords)
                 all_folder_names.append(folder)
            elif tx_coords.ndim == 2 and tx_coords.shape == (1, 3):
                 all_coords_list.append(tx_coords.flatten()) # Flatten if shape is (1, 3)
                 all_folder_names.append(folder)
            else:
                print(f"  Warning: Skipping {folder}. Unexpected coordinate shape: {tx_coords.shape}")

        else:
            print(f"  Warning: Key '{key_name}' not found in {file_path}. Skipping folder.")
    except FileNotFoundError:
        print(f"  Warning: File not found: {file_path}. Skipping folder.")
    except Exception as e:
        print(f"  Warning: Error processing {folder}: {e}. Skipping folder.")

if len(all_coords_list) < num_points_to_select:
    print(f"\nError: Only found {len(all_coords_list)} valid coordinates. Cannot select {num_points_to_select}.")
else:
    all_coords = np.array(all_coords_list)
    num_total_points = all_coords.shape[0]
    print(f"\nLoaded {num_total_points} coordinates.")

    selected_indices = []
    remaining_indices = list(range(num_total_points))

    # --- 2. Iterative Farthest Point Selection ---
    # Start with the first point (or could be random)
    start_index = 0
    selected_indices.append(start_index)
    remaining_indices.pop(start_index)
    print(f"Selected point 1: Index {start_index} (Folder: {all_folder_names[start_index]}) Coords: {all_coords[start_index]}")


    for i in range(1, num_points_to_select):
        # Coordinates of already selected points
        selected_coords = all_coords[selected_indices]
        # Coordinates of remaining points
        remaining_coords = all_coords[remaining_indices]

        # Calculate distances from each remaining point to *all* selected points
        # dists will have shape (num_remaining, num_selected)
        dists = pairwise_distances(remaining_coords, selected_coords)

        # Find the minimum distance from each remaining point to any selected point
        # min_dists will have shape (num_remaining,)
        min_dists = np.min(dists, axis=1)

        # Find the index *within remaining_indices* of the point with the maximum minimum distance
        farthest_remaining_idx = np.argmax(min_dists)

        # Get the original index of that point
        farthest_original_idx = remaining_indices[farthest_remaining_idx]

        # Add to selected list and remove from remaining list
        selected_indices.append(farthest_original_idx)
        remaining_indices.pop(farthest_remaining_idx) # Remove by index within remaining_indices

        print(f"Selected point {i+1}: Index {farthest_original_idx} (Folder: {all_folder_names[farthest_original_idx]}) Coords: {all_coords[farthest_original_idx]}")


    # --- 3. Display Results ---
    print("\n--- Final {num_points_to_select} diverse Tx points: ---")
    selected_final_coords = all_coords[selected_indices]
    selected_final_folders = [all_folder_names[i] for i in selected_indices]

    for i in range(num_points_to_select):
        print(f"  Folder: {selected_final_folders[i]}, Coords: {selected_final_coords[i]}")

--- Reading Tx coordinates and selecting 3 diverse points ---

Loaded 15 coordinates.
Selected point 1: Index 0 (Folder: tx_sphere_Tx_1) Coords: [4342.70852986 2714.91536661   42.7       ]
Selected point 2: Index 7 (Folder: tx_sphere_Tx_18) Coords: [5680.98977986 5196.16536661   83.9       ]
Selected point 3: Index 14 (Folder: tx_sphere_Tx_8) Coords: [4311.33352986 4357.16536661   70.3       ]

--- Final {num_points_to_select} diverse Tx points: ---
  Folder: tx_sphere_Tx_1, Coords: [4342.70852986 2714.91536661   42.7       ]
  Folder: tx_sphere_Tx_18, Coords: [5680.98977986 5196.16536661   83.9       ]
  Folder: tx_sphere_Tx_8, Coords: [4311.33352986 4357.16536661   70.3       ]


In [ ]:
Tx_subset_selected = [1,19,18,8,2]

In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # Use only GPU 1

# --- Rest of your imports ---
import torch
# ... other imports

# --- Setup ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Old way
# The line above will now automatically select the visible GPU (GPU 1) if cuda is available
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') # Explicitly use the *first visible* GPU
print(f"Using device: {device}")
# ... rest of script ...

Using device: cuda:1


In [35]:
import random
from pathlib import Path
import sys

# --- Configuration ---
BASE_DIR = Path('/home/mkrishne/PL_competition/extracted_regions/train')  # Assumes you run this script from the parent directory
FOLDER_PATTERN = '*_processed__for_training_Tx_*' # Pattern to find the target folders
FILE_PATTERN = 'pair*_for_train.npy' # Pattern for the individual data files
OUTPUT_FILENAME = 'random_subset' # The new file to be created in each folder
SAMPLE_SIZE = 8000
# ---------------------

def create_random_subset_from_filenames():
    """
    Finds all processed folders, scans for .npy files, and creates a
    'random_subset' file in each containing a random sample of pair IDs.
    """
    print(f"Starting script in: {BASE_DIR.resolve()}")
    
    # 1. Find all target folders
    target_folders = [d for d in BASE_DIR.glob(FOLDER_PATTERN) if d.is_dir()]
    
    if not target_folders:
        print(f"Error: No folders found matching pattern '{FOLDER_PATTERN}'.")
        print("Please run this script from the directory containing all the 'freq_processed...' folders.")
        return

    print(f"Found {len(target_folders)} matching folders. Processing...")

    for folder_path in target_folders:
        print(f"\n--- Processing: {folder_path.name} ---")

        # 2. Find all .npy files matching the pattern
        # We use .glob() to find the files and .stem to get the filename without .npy
        # e.g., 'pair00000_for_train.npy' -> 'pair00000_for_train'
        # We must adjust this if the ID is just 'pair00000'
        
        all_pair_files = list(folder_path.glob(FILE_PATTERN))
        
        if not all_pair_files:
            print(f"  [WARNING] No files found matching '{FILE_PATTERN}' in '{folder_path}'. Skipping.")
            continue
            
        # 3. Extract the pair ID from the filename.
        # We assume the ID is the filename without the '_for_train.npy' part
        # Example: 'pair00000_for_train.npy' -> 'pair00000'
        all_pair_ids_list = []
        for f_path in all_pair_files:
            # f_path.stem is 'pair00000_for_train'
            # We split at the known suffix '_for_train'
            pair_id = f_path.stem.split('_for_train')[0]
            all_pair_ids_list.append(pair_id)

        num_ids = len(all_pair_ids_list)
        print(f"  Found {num_ids} total pair files (e.g., '{all_pair_ids_list[0]}').")

        # 4. Randomly sample 8,000 IDs
        if num_ids == 0:
            print("  [WARNING] No IDs to sample. Creating an empty file.")
            subset_ids = []
        elif num_ids < SAMPLE_SIZE:
            print(f"  [WARNING] Only {num_ids} IDs available (less than {SAMPLE_SIZE}). Using all of them.")
            subset_ids = all_pair_ids_list
            random.shuffle(subset_ids) # Still randomize the order
        else:
            subset_ids = random.sample(all_pair_ids_list, SAMPLE_SIZE)
        
        # 5. & 6. Write the IDs to the new 'random_subset' file
        output_file_path = folder_path / OUTPUT_FILENAME
        with open(output_file_path, 'w') as f:
            for pid in subset_ids:
                f.write(f"{pid}\n") # Write one ID per line

        print(f"  Successfully created '{output_file_path.name}' with {len(subset_ids)} IDs.")

    print("\n--- Script finished ---")

if __name__ == "__main__":
    create_random_subset_from_filenames()

Starting script in: /home/mkrishne/PL_competition/extracted_regions/train
Found 45 matching folders. Processing...

--- Processing: 7GHz_processed__for_training_Tx_1 ---
  Found 29551 total pair files (e.g., 'pair00000').
  Successfully created 'random_subset' with 8000 IDs.

--- Processing: 7GHz_processed__for_training_Tx_10 ---
  Found 26167 total pair files (e.g., 'pair00000').
  Successfully created 'random_subset' with 8000 IDs.

--- Processing: 7GHz_processed__for_training_Tx_11 ---
  Found 25263 total pair files (e.g., 'pair00000').
  Successfully created 'random_subset' with 8000 IDs.

--- Processing: 7GHz_processed__for_training_Tx_13 ---
  Found 24098 total pair files (e.g., 'pair00000').
  Successfully created 'random_subset' with 8000 IDs.

--- Processing: 7GHz_processed__for_training_Tx_15 ---
  Found 22348 total pair files (e.g., 'pair00000').
  Successfully created 'random_subset' with 8000 IDs.

--- Processing: 7GHz_processed__for_training_Tx_16 ---
  Found 22245 total 

In [29]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random # Needed for sampling

# --- 1. Configuration ---

# Base directory for all ..._for_training_Tx_... folders
BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- Checkpoint Configuration ---
MODEL_SAVE_PATH = "pointnet_dual_stream_model.pth" # New save path
# ... (rest of config) ...
LEARNING_RATE = 0.01
NUM_EPOCHS = 10
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
NUM_SAMPLES_PER_FOLDER = 8000
FEATURE_DIM = 1536 # This will be the output of the encoder

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7GHz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED. Check folder names and IDs.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------


# --- 2. Standard PyTorch Dataset (Same as before) ---
class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir, num_samples_per_folder=None):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for files...")
        total_potential_files = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            potential_files_in_folder = []
            input_files = sorted(glob.glob(os.path.join(folder_path, "pair*_for_train.npy")))

            for input_path in input_files:
                filename = os.path.basename(input_path)
                pair_id_str = filename.split('_')[0].replace('pair', '')
                label_filename = f"pair{pair_id_str}_path_loss.npy"
                label_path = os.path.join(folder_path, label_filename)

                if os.path.exists(label_path):
                    potential_files_in_folder.append((input_path, label_path))

            num_found_in_folder = len(potential_files_in_folder)
            total_potential_files += num_found_in_folder

            if num_samples_per_folder is not None and num_found_in_folder > num_samples_per_folder:
                sampled_files_for_folder = random.sample(potential_files_in_folder, num_samples_per_folder)
                self.file_pairs.extend(sampled_files_for_folder)
            elif num_samples_per_folder is not None and num_found_in_folder < num_samples_per_folder:
                 print(f"  Warning: Found only {num_found_in_folder} files in {folder_name}, less than requested {num_samples_per_folder}. Using all found.")
                 self.file_pairs.extend(potential_files_in_folder)
            else:
                self.file_pairs.extend(potential_files_in_folder)

        print(f"\nFound {total_potential_files} total potential files across all folders.")
        print(f"Using {len(self.file_pairs)} files for training after sampling.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32)
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor = torch.from_numpy(label).squeeze()
        return pc_tensor, label_tensor

# --- 3. MODIFIED: Dual-Stream PointNet Model ---

class PointNetEncoder(nn.Module):
    """
    Modified PointNet encoder.
    Processes (x,y,z) coordinates and (other) features in separate streams
    before combining them. This is closer to the PointNet++ concept.
    """
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        
        # --- Stream 1: Process Coordinates (x,y,z) ---
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)

        # --- Stream 2: Process Features (is_bldg, f1, f2, f3) ---
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        
        # --- Combined Stream ---
        # Takes the concatenated features (128 from pos + 128 from feat = 256)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        # Input x: (batch, input_dim, num_points), e.g., (batch, 7, 7000)
        
        # Split the input into coordinates and features
        pos_stream = x[:, :3, :] # (batch, 3, 7000)
        feat_stream = x[:, 3:, :] # (batch, 4, 7000)

        # Process coordinates
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out))) # (batch, 128, 7000)
        
        # Process features
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out))) # (batch, 128, 7000)

        # Combine the processed streams
        combined = torch.cat([pos_out, feat_out], dim=1) # (batch, 256, 7000)

        # Process the combined features
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) # (batch, feature_dim_out, 7000)

        # Global max pooling
        x = torch.max(x, 2)[0]  # (batch, feature_dim_out)
        return x

class PointNetPathLoss(nn.Module):
    """PointNet for path loss prediction (Regression Head)"""
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        # Use the new Dual-Stream Encoder
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # Regression head
        self.fc1 = nn.Linear(feature_dim_encoder, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Input x: (batch, num_points, input_dim), e.g., (batch, 7000, 7)
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        
        # Encoder processes the 7D input, splitting it internally
        x = self.encoder(x) # -> (batch, feature_dim_encoder)

        # Regression head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x.squeeze(-1)

# --- 4. Main Training Script ---

if __name__ == "__main__":

    # --- Setup ---
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    print(f"Starting training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR,
        num_samples_per_folder=NUM_SAMPLES_PER_FOLDER
    )

    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty after filtering and sampling. Check paths and configuration.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # This now uses the Dual-Stream model

    # Calculate Model Size
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Dual-Stream Encoder)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Total Trainable Parameters: {total_params:,}")

    # --- Load existing MODEL WEIGHTS if they exist ---
    start_epoch = 0
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing model weights from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            # Use strict=False, as the architecture has changed from the previous model
            model.load_state_dict(checkpoint['model_state_dict'], strict=False) 
            print("Model weights loaded (non-strictly).")
            # Check if optimizer and epoch can be loaded (they probably won't match)
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                 # optimizer.load_state_dict(checkpoint['optimizer_state_dict']) # Better to start optimizer fresh with new arch
                 # start_epoch = checkpoint['epoch'] + 1
                 print("Starting from epoch 0 with fresh optimizer due to arch change.")
            else:
                print("Starting from epoch 0.")

        except Exception as e:
            print(f"Error loading model state dict: {e}. Starting fresh from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")

    # --- Initialize Loss and Optimizer ---
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            out = model(pc_batch)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)

        if len(train_loader.dataset) > 0:
             avg_train_loss = total_train_loss / len(train_loader.dataset)
             avg_train_rmse = np.sqrt(max(0, avg_train_loss))
             print(f"Epoch {epoch+1:02d} | Train RMSE: {avg_train_rmse:8.4f}")
        else:
             print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_train_loss,
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")
    print(f"Finished training. Final model state saved at {MODEL_SAVE_PATH}")

Physical Batch Size: 32
Filtered to 15 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda:1
Starting training with LEARNING_RATE = 0.01 for 10 epochs.
Scanning 15 folders for files...


Scanning folders: 100%|██████████| 15/15 [00:01<00:00,  7.52it/s]



Found 372148 total potential files across all folders.
Using 120000 files for training after sampling.
Model Initialized: PointNetPathLoss (Dual-Stream Encoder)
Target Encoder Output Dimension: 1536
Total Trainable Parameters: 3,175,489
No existing model found at pointnet_dual_stream_model.pth. Starting fresh from epoch 0.

--- Starting Training ---


Epoch 1/10 [Train]: 100%|██████████| 3750/3750 [11:32<00:00,  5.41it/s]


Epoch 01 | Train RMSE:  21.8341
Saving model checkpoint after epoch 1...
Model saved to pointnet_dual_stream_model.pth


...

Epoch 08 | Train RMSE:  13.9100
Saving model checkpoint after epoch 8...
Model saved to pointnet_dual_stream_model.pth


Epoch 9/10 [Train]: 100%|██████████| 3750/3750 [11:29<00:00,  5.43it/s]


Epoch 09 | Train RMSE:  13.6960
Saving model checkpoint after epoch 9...
Model saved to pointnet_dual_stream_model.pth


Epoch 10/10 [Train]: 100%|██████████| 3750/3750 [11:30<00:00,  5.43it/s]

Epoch 10 | Train RMSE:  13.5521
Saving model checkpoint after epoch 10...
Model saved to pointnet_dual_stream_model.pth
--- Training Complete ---
Finished training. Final model state saved at pointnet_dual_stream_model.pth


In [46]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random # Needed for sampling

# --- 1. Configuration ---

# Base directory for all ..._for_training_Tx_... folders
BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- Checkpoint Configuration ---
MODEL_SAVE_PATH = "pointnet_dual_stream_model_1536.pth" # New save path
# ... (rest of config) ...
LEARNING_RATE = 0.01
NUM_EPOCHS = 20
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
# NUM_SAMPLES_PER_FOLDER = 8000 # <-- This is no longer needed, we use 'random_subset'
FEATURE_DIM = 1536 # This will be the output of the encoder

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED. Check folder names and IDs.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------


# --- 2. MODIFIED: Dataset reads from 'random_subset' ---
class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            # This is the file we created in the previous step
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                # Read all pair IDs from the subset file
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() # Remove newline characters (e.g., "pair00000\n")
                    if not pair_id: # Skip any empty lines
                        continue
                        
                    # Construct filenames from the ID
                    # e.g., pair_id = "pair00123"
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    # Check if both files actually exist before adding
                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        # This warning is important, in case 'random_subset' is stale
                        print(f"\n  Warning: Files for '{pair_id}' listed in 'random_subset' but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")


    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32)
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor = torch.from_numpy(label).squeeze()
        return pc_tensor, label_tensor

# --- 3. MODIFIED: Dual-Stream PointNet Model ---

class PointNetEncoder(nn.Module):
    """
    Modified PointNet encoder.
    Processes (x,y,z) coordinates and (other) features in separate streams
    before combining them. This is closer to the PointNet++ concept.
    """
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        
        # --- Stream 1: Process Coordinates (x,y,z) ---
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)

        # --- Stream 2: Process Features (is_bldg, f1, f2, f3) ---
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        
        # --- Combined Stream ---
        # Takes the concatenated features (128 from pos + 128 from feat = 256)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        # Input x: (batch, input_dim, num_points), e.g., (batch, 7, 7000)
        
        # Split the input into coordinates and features
        pos_stream = x[:, :3, :] # (batch, 3, 7000)
        feat_stream = x[:, 3:, :] # (batch, 4, 7000)

        # Process coordinates
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out))) # (batch, 128, 7000)
        
        # Process features
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out))) # (batch, 128, 7000)

        # Combine the processed streams
        combined = torch.cat([pos_out, feat_out], dim=1) # (batch, 256, 7000)

        # Process the combined features
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) # (batch, feature_dim_out, 7000)

        # Global max pooling
        x = torch.max(x, 2)[0]  # (batch, feature_dim_out)
        return x

class PointNetPathLoss(nn.Module):
    """PointNet for path loss prediction (Regression Head)"""
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        # Use the new Dual-Stream Encoder
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # Regression head
        self.fc1 = nn.Linear(feature_dim_encoder, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Input x: (batch, num_points, input_dim), e.g., (batch, 7000, 7)
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        
        # Encoder processes the 7D input, splitting it internally
        x = self.encoder(x) # -> (batch, feature_dim_encoder)

        # Regression head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x.squeeze(-1)

# --- 4. Main Training Script (WITH CHECKPOINT FIX) ---

if __name__ == "__main__":

    # --- Setup ---
    # *** THIS IS THE FIX ***
    # Use 'cuda' to automatically select the available GPU (e.g., cuda:0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    # *** END FIX ***
    
    print(f"Starting training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
        # No 'num_samples_per_folder' needed, it's now controlled by 'random_subset'
    )

    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty after filtering and loading from 'random_subset' files. Check paths and files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # This now uses the Dual-Stream model

    # Calculate Model Size
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Dual-Stream Encoder)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Total Trainable Parameters: {total_params:,}")
    
    # --- Initialize Loss and Optimizer (MOVED UP) ---
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # --------------------------------------------------

    # --- Load existing MODEL and OPTIMIZER states ---
    start_epoch = 0
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            # The 'map_location=device' handles moving the checkpoint
            # from one GPU (e.g., cuda:1) to another (e.g., cuda:0)
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Load model weights (use strict=True if architecture is stable)
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully.")
            
            # Check if optimizer and epoch are in the checkpoint
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
                print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
                
                # Check if we've already finished training
                if start_epoch >= NUM_EPOCHS:
                    print(f"Warning: Model has already completed {start_epoch} epochs. ")
                    print(f"         NUM_EPOCHS is set to {NUM_EPOCHS}. Increase NUM_EPOCHS to train further.")
            else:
                print("Optimizer state/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            print(f"Error loading checkpoint: {e}. Starting fresh from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    # Only train if start_epoch is less than NUM_EPOCHS
    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        avg_train_loss = 0 # Initialize here

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            out = model(pc_batch)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)

        if len(train_loader.dataset) > 0:
            avg_train_loss = total_train_loss / len(train_loader.dataset)
            avg_train_rmse = np.sqrt(max(0, avg_train_loss))
            print(f"Epoch {epoch+1:02d} | Train RMSE: {avg_train_rmse:8.4f}")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_train_loss, # Save the average loss
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")
    if start_epoch >= NUM_EPOCHS:
         print(f"Training was already complete at {start_epoch} epochs.")
    else:
        print(f"Finished training. Final model state")

Physical Batch Size: 32
Filtered to 15 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting training with LEARNING_RATE = 0.01 for 20 epochs.
Scanning 15 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 15/15 [00:00<00:00, 19.87it/s]
/tmp/ipykernel_87127/2284511277.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.lo


Loaded a total of 120000 file pairs from all 'random_subset' files.
Total dataset size: 120000 files.
Model Initialized: PointNetPathLoss (Dual-Stream Encoder)
Target Encoder Output Dimension: 1536
Total Trainable Parameters: 3,175,489
Loading existing checkpoint from pointnet_dual_stream_model_1536.pth
Model weights loaded successfully.
Optimizer state loaded. Resuming training from epoch 16.

--- Starting Training ---


Epoch 16/20 [Train]: 100%|██████████| 3750/3750 [05:10<00:00, 12.08it/s]


Epoch 16 | Train RMSE:  12.7749
Saving model checkpoint after epoch 16...
Model saved to pointnet_dual_stream_model_1536.pth


...

Epoch 18 | Train RMSE:  12.6105
Saving model checkpoint after epoch 18...
Model saved to pointnet_dual_stream_model_1536.pth


Epoch 19/20 [Train]: 100%|██████████| 3750/3750 [05:08<00:00, 12.14it/s]


Epoch 19 | Train RMSE:  12.5234
Saving model checkpoint after epoch 19...
Model saved to pointnet_dual_stream_model_1536.pth


Epoch 20/20 [Train]: 100%|██████████| 3750/3750 [05:09<00:00, 12.13it/s]


Epoch 20 | Train RMSE:  12.3919
Saving model checkpoint after epoch 20...
Model saved to pointnet_dual_stream_model_1536.pth
--- Training Complete ---
Finished training. Final model state


In [48]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random # Needed for sampling

# --- 1. Configuration ---

# Base directory for all ..._for_training_Tx_... folders
BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- Checkpoint Configuration ---
MODEL_SAVE_PATH = "pointnet_dual_stream_model_1536.pth" # New save path
# ... (rest of config) ...
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
# NUM_SAMPLES_PER_FOLDER = 8000 # <-- This is no longer needed, we use 'random_subset'
FEATURE_DIM = 1536 # This will be the output of the encoder

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED. Check folder names and IDs.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------


# --- 2. MODIFIED: Dataset reads from 'random_subset' ---
class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            # This is the file we created in the previous step
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                # Read all pair IDs from the subset file
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() # Remove newline characters (e.g., "pair00000\n")
                    if not pair_id: # Skip any empty lines
                        continue
                        
                    # Construct filenames from the ID
                    # e.g., pair_id = "pair00123"
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    # Check if both files actually exist before adding
                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        # This warning is important, in case 'random_subset' is stale
                        print(f"\n  Warning: Files for '{pair_id}' listed in 'random_subset' but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")


    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32)
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor = torch.from_numpy(label).squeeze()
        return pc_tensor, label_tensor

# --- 3. MODIFIED: Dual-Stream PointNet Model ---

class PointNetEncoder(nn.Module):
    """
    Modified PointNet encoder.
    Processes (x,y,z) coordinates and (other) features in separate streams
    before combining them. This is closer to the PointNet++ concept.
    """
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        
        # --- Stream 1: Process Coordinates (x,y,z) ---
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)

        # --- Stream 2: Process Features (is_bldg, f1, f2, f3) ---
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        
        # --- Combined Stream ---
        # Takes the concatenated features (128 from pos + 128 from feat = 256)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        # Input x: (batch, input_dim, num_points), e.g., (batch, 7, 7000)
        
        # Split the input into coordinates and features
        pos_stream = x[:, :3, :] # (batch, 3, 7000)
        feat_stream = x[:, 3:, :] # (batch, 4, 7000)

        # Process coordinates
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out))) # (batch, 128, 7000)
        
        # Process features
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out))) # (batch, 128, 7000)

        # Combine the processed streams
        combined = torch.cat([pos_out, feat_out], dim=1) # (batch, 256, 7000)

        # Process the combined features
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) # (batch, feature_dim_out, 7000)

        # Global max pooling
        x = torch.max(x, 2)[0]  # (batch, feature_dim_out)
        return x

class PointNetPathLoss(nn.Module):
    """PointNet for path loss prediction (Regression Head)"""
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        # Use the new Dual-Stream Encoder
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # Regression head
        self.fc1 = nn.Linear(feature_dim_encoder, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Input x: (batch, num_points, input_dim), e.g., (batch, 7000, 7)
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        
        # Encoder processes the 7D input, splitting it internally
        x = self.encoder(x) # -> (batch, feature_dim_encoder)

        # Regression head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x.squeeze(-1)

# --- 4. Main Training Script (WITH CHECKPOINT FIX) ---

if __name__ == "__main__":

    # --- Setup ---
    # *** THIS IS THE FIX ***
    # Use 'cuda' to automatically select the available GPU (e.g., cuda:0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    # *** END FIX ***
    
    print(f"Starting training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
        # No 'num_samples_per_folder' needed, it's now controlled by 'random_subset'
    )

    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty after filtering and loading from 'random_subset' files. Check paths and files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # This now uses the Dual-Stream model

    # Calculate Model Size
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Dual-Stream Encoder)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Total Trainable Parameters: {total_params:,}")
    
    # --- Initialize Loss and Optimizer (MOVED UP) ---
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # --------------------------------------------------

    # --- Load existing MODEL and OPTIMIZER states ---
    start_epoch = 0
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            # The 'map_location=device' handles moving the checkpoint
            # from one GPU (e.g., cuda:1) to another (e.g., cuda:0)
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Load model weights (use strict=True if architecture is stable)
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully.")
            
            # Check if optimizer and epoch are in the checkpoint
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
                print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
                
                # Check if we've already finished training
                if start_epoch >= NUM_EPOCHS:
                    print(f"Warning: Model has already completed {start_epoch} epochs. ")
                    print(f"         NUM_EPOCHS is set to {NUM_EPOCHS}. Increase NUM_EPOCHS to train further.")
            else:
                print("Optimizer state/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            print(f"Error loading checkpoint: {e}. Starting fresh from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    # Only train if start_epoch is less than NUM_EPOCHS
    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        avg_train_loss = 0 # Initialize here

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            out = model(pc_batch)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)

        if len(train_loader.dataset) > 0:
            avg_train_loss = total_train_loss / len(train_loader.dataset)
            avg_train_rmse = np.sqrt(max(0, avg_train_loss))
            print(f"Epoch {epoch+1:02d} | Train RMSE: {avg_train_rmse:8.4f}")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_train_loss, # Save the average loss
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")
    if start_epoch >= NUM_EPOCHS:
         print(f"Training was already complete at {start_epoch} epochs.")
    else:
        print(f"Finished training. Final model state")

Physical Batch Size: 32
Filtered to 15 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting training with LEARNING_RATE = 0.001 for 50 epochs.
Scanning 15 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 15/15 [00:00<00:00, 20.75it/s]
/tmp/ipykernel_87127/1862504709.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.lo


Loaded a total of 120000 file pairs from all 'random_subset' files.
Total dataset size: 120000 files.
Model Initialized: PointNetPathLoss (Dual-Stream Encoder)
Target Encoder Output Dimension: 1536
Total Trainable Parameters: 3,175,489
Loading existing checkpoint from pointnet_dual_stream_model_1536.pth
Model weights loaded successfully.
Optimizer state loaded. Resuming training from epoch 31.

--- Starting Training ---


Epoch 31/50 [Train]: 100%|██████████| 3750/3750 [05:09<00:00, 12.13it/s]


Epoch 31 | Train RMSE:  11.8113
Saving model checkpoint after epoch 31...
Model saved to pointnet_dual_stream_model_1536.pth


...

Epoch 48 | Train RMSE:  11.2911
Saving model checkpoint after epoch 48...
Model saved to pointnet_dual_stream_model_1536.pth


Epoch 49/50 [Train]: 100%|██████████| 3750/3750 [05:09<00:00, 12.13it/s]


Epoch 49 | Train RMSE:  11.2838
Saving model checkpoint after epoch 49...
Model saved to pointnet_dual_stream_model_1536.pth


Epoch 50/50 [Train]: 100%|██████████| 3750/3750 [05:08<00:00, 12.14it/s]


Epoch 50 | Train RMSE:  11.2461
Saving model checkpoint after epoch 50...
Model saved to pointnet_dual_stream_model_1536.pth
--- Training Complete ---
Finished training. Final model state


In [51]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re # Needed for parsing folder names

# --- 1. Configuration ---

# Base directory for all ..._for_training_Tx_... folders
BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
# Directory to save the resulting CSVs
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- Checkpoint Configuration ---
MODEL_SAVE_PATH = "pointnet_dual_stream_model.pth"
FEATURE_DIM = 1536
BATCH_SIZE = 32 # Can be larger for evaluation if VRAM allows
NUM_WORKERS = 4

# --- Test Set Configuration ---
TX_SUBSET_SELECTED = [5, 9, 12, 14, 20]
TX_SUBSET_SELECTED = [9, 12, 14, 20]
# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_12",
    "28GHz_processed__for_training_Tx_14",
    "28GHz_processed__for_training_Tx_20",
    "28GHz_processed__for_training_Tx_5",
    "28GHz_processed__for_training_Tx_9",
    "7GHz_processed__for_training_Tx_12",
    "7GHz_processed__for_training_Tx_14",
    "7GHz_processed__for_training_Tx_20",
    "7GHz_processed__for_training_Tx_5",
    "7GHz_processed__for_training_Tx_9",
    "800MHz_processed__for_training_Tx_12",
    "800MHz_processed__for_training_Tx_14",
    "800MHz_processed__for_training_Tx_20",
    "800MHz_processed__for_training_Tx_5",
    "800MHz_processed__for_training_Tx_9"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not TEST_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------


# --- 2. Dataset Definition for Evaluation ---
# This dataset loads ALL files, not just a 'random_subset'
class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        # Find all input files in the folder
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            # Construct the corresponding label file path
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
                
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32)
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor = torch.from_numpy(label).squeeze()
        return pc_tensor, label_tensor

# --- 3. Model Definitions (Copied from your training script) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0]
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x.squeeze(-1)

# --- 4. Main Evaluation Script ---

if __name__ == "__main__":

    # --- Setup ---
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device)
    print("Model initialized.")

    # --- Load Checkpoint ---
    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}. Cannot evaluate.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    # Set model to evaluation mode (disables dropout, batchnorm updates)
    model.eval()
    criterion = torch.nn.MSELoss() # For calculating RMSE

    # Lists to store results for overall RMSE
    all_predictions = []
    all_labels = []

    print("\n--- Starting Evaluation ---")
    
    # Loop through each folder in the test set
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        # --- Create Dataset and Dataloader ---
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False, # No need to shuffle for evaluation
            num_workers=NUM_WORKERS,
            pin_memory=True if device.type == 'cuda' else False
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_labels_list = []

        # --- Run Inference ---
        with torch.no_grad(): # Disables gradient calculation
            for pc_batch, label_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_batch = label_batch.to(device)
                
                out = model(pc_batch)
                
                # Store results
                folder_preds_list.append(out.cpu().numpy())
                folder_labels_list.append(label_batch.cpu().numpy())
        
        # --- Process and Save Results for this Folder ---
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels = np.concatenate(folder_labels_list)
        
        # Add to overall lists for final RMSE
        all_predictions.append(predictions)
        all_labels.append(labels)
        
        # Calculate folder-specific RMSE
        folder_mse = np.mean((predictions - labels)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE: {folder_rmse:8.4f}")
        
        # --- Create and Save CSV ---
        # Parse freq and tx_id from folder name
        # e.g., "800MHz_processed__for_training_Tx_5"
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'actual_pathloss': labels
        })
        
        # Save to CSV
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions:
        overall_preds = np.concatenate(all_predictions)
        overall_labels = np.concatenate(all_labels)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE: {overall_rmse:8.4f}")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 12 folders for evaluation for Tx IDs: [9, 12, 14, 20]
Using device: cuda
Model initialized.
Successfully loaded model weights from pointnet_dual_stream_model.pth

--- Starting Evaluation ---

▶ Processing folder: 28GHz_processed__for_training_Tx_12


/tmp/ipykernel_87127/901120766.py:168: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|███████

  -> Folder RMSE:  26.5028
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_12.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_14


  Evaluating: 100%|██████████| 701/701 [00:20<00:00, 34.77it/s]


  -> Folder RMSE:  19.1019
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_14.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_20


...

  -> Folder RMSE:  14.4491
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_14.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_20


  Evaluating: 100%|██████████| 696/696 [00:20<00:00, 34.71it/s]


  -> Folder RMSE:  16.4006
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_20.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_9


  Evaluating: 100%|██████████| 762/762 [00:21<00:00, 34.75it/s]


  -> Folder RMSE:  17.3675
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_9.csv

--- Evaluation Complete ---
Overall Test RMSE:  20.1140


In [52]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re # Needed for parsing folder names

# --- 1. Configuration ---

# Base directory for all ..._for_training_Tx_... folders
BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
# Directory to save the resulting CSVs
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- Checkpoint Configuration ---
MODEL_SAVE_PATH = "pointnet_dual_stream_model_1536.pth"
FEATURE_DIM = 1536
BATCH_SIZE = 32 # Can be larger for evaluation if VRAM allows
NUM_WORKERS = 4

# --- Test Set Configuration ---
TX_SUBSET_SELECTED = [5, 9, 12, 14, 20]
TX_SUBSET_SELECTED = [9, 12, 14, 20]
# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_12",
    "28GHz_processed__for_training_Tx_14",
    "28GHz_processed__for_training_Tx_20",
    "28GHz_processed__for_training_Tx_5",
    "28GHz_processed__for_training_Tx_9",
    "7GHz_processed__for_training_Tx_12",
    "7GHz_processed__for_training_Tx_14",
    "7GHz_processed__for_training_Tx_20",
    "7GHz_processed__for_training_Tx_5",
    "7GHz_processed__for_training_Tx_9",
    "800MHz_processed__for_training_Tx_12",
    "800MHz_processed__for_training_Tx_14",
    "800MHz_processed__for_training_Tx_20",
    "800MHz_processed__for_training_Tx_5",
    "800MHz_processed__for_training_Tx_9"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not TEST_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------


# --- 2. Dataset Definition for Evaluation ---
# This dataset loads ALL files, not just a 'random_subset'
class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        # Find all input files in the folder
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            # Construct the corresponding label file path
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
                
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32)
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor = torch.from_numpy(label).squeeze()
        return pc_tensor, label_tensor

# --- 3. Model Definitions (Copied from your training script) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0]
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x.squeeze(-1)

# --- 4. Main Evaluation Script ---

if __name__ == "__main__":

    # --- Setup ---
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device)
    print("Model initialized.")

    # --- Load Checkpoint ---
    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}. Cannot evaluate.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    # Set model to evaluation mode (disables dropout, batchnorm updates)
    model.eval()
    criterion = torch.nn.MSELoss() # For calculating RMSE

    # Lists to store results for overall RMSE
    all_predictions = []
    all_labels = []

    print("\n--- Starting Evaluation ---")
    
    # Loop through each folder in the test set
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        # --- Create Dataset and Dataloader ---
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False, # No need to shuffle for evaluation
            num_workers=NUM_WORKERS,
            pin_memory=True if device.type == 'cuda' else False
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_labels_list = []

        # --- Run Inference ---
        with torch.no_grad(): # Disables gradient calculation
            for pc_batch, label_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_batch = label_batch.to(device)
                
                out = model(pc_batch)
                
                # Store results
                folder_preds_list.append(out.cpu().numpy())
                folder_labels_list.append(label_batch.cpu().numpy())
        
        # --- Process and Save Results for this Folder ---
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels = np.concatenate(folder_labels_list)
        
        # Add to overall lists for final RMSE
        all_predictions.append(predictions)
        all_labels.append(labels)
        
        # Calculate folder-specific RMSE
        folder_mse = np.mean((predictions - labels)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE: {folder_rmse:8.4f}")
        
        # --- Create and Save CSV ---
        # Parse freq and tx_id from folder name
        # e.g., "800MHz_processed__for_training_Tx_5"
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'actual_pathloss': labels
        })
        
        # Save to CSV
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions:
        overall_preds = np.concatenate(all_predictions)
        overall_labels = np.concatenate(all_labels)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE: {overall_rmse:8.4f}")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 12 folders for evaluation for Tx IDs: [9, 12, 14, 20]
Using device: cuda
Model initialized.


/tmp/ipykernel_87127/2430055321.py:168: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)


Successfully loaded model weights from pointnet_dual_stream_model_1536.pth

--- Starting Evaluation ---

▶ Processing folder: 28GHz_processed__for_training_Tx_12


  Evaluating: 100%|██████████| 729/729 [00:47<00:00, 15.45it/s]


  -> Folder RMSE:  26.2872
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_12.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_14


...

  -> Folder RMSE:  15.3865
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_14.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_20


  Evaluating: 100%|██████████| 696/696 [00:45<00:00, 15.46it/s]


  -> Folder RMSE:  18.9718
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_20.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_9


  Evaluating: 100%|██████████| 762/762 [00:49<00:00, 15.48it/s]

  -> Folder RMSE:  18.9048
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_9.csv

--- Evaluation Complete ---
Overall Test RMSE:  20.5996


In [54]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- MODIFIED: Checkpoint, Feature Dim, and Class Config ---
FEATURE_DIM = 512 # REDUCED from 1536 to lower parameter count
NUM_CLASSES = 16  # (200 - 50) / 10 + 1 = 16 classes (50, 60, ..., 200)
MODEL_SAVE_PATH = "pointnet_classifier_model_800.pth" # New path for the new model

LEARNING_RATE = 0.001
NUM_EPOCHS = 20
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. MODIFIED: Dataset handles classification ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index."""
    # 1. Round to nearest 10
    pl_rounded = torch.round(pl_tensor / 10.0) * 10.0
    # 2. Clip to the valid range [50, 200]
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    # 3. Convert to class index: (50 -> 0, 60 -> 1, ..., 200 -> 15)
    class_index = (pl_clipped - 50.0) / 10.0
    # 4. Return as a long integer for CrossEntropyLoss
    return class_index.long()

class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() 
                    if not pair_id: 
                        continue
                        
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        print(f"\n  Warning: Files for '{pair_id}' listed but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32) # Load as float
        
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor_float = torch.from_numpy(label).squeeze()
        
        # --- MODIFIED: Convert float label to class index ---
        label_class_tensor = pl_to_class(label_tensor_float)
        
        return pc_tensor, label_class_tensor # Return class index

# --- 3. MODIFIED: Model head reduced and outputting 16 classes ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        # (This part is unchanged, but feature_dim_out is now 512)
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> (batch, 512, 7000)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :] 
        feat_stream = x[:, 3:, :] 
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1) 
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) 
        x = torch.max(x, 2)[0]  # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # --- MODIFIED: Reduced Regression head for ~1M params & classification ---
        self.fc1 = nn.Linear(feature_dim_encoder, 512) # 512 -> 512
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # Output 16 classes

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        x = self.encoder(x)    # -> (batch, 512)

        # Classification head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 16)
        return x # Do not squeeze

# --- 4. Main Training Script (Modified for Classification) ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    print(f"Starting classification training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
    )
    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty. Check paths and 'random_subset' files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # Model head is now for classification

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Classifier)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Total Trainable Parameters: {total_params:,}") # ~838k
    
    # --- MODIFIED: Loss and Optimizer ---
    criterion = torch.nn.CrossEntropyLoss() # Switched to CrossEntropy for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # ------------------------------------

    # --- Load Checkpoint ---
    start_epoch = 0
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Use strict=False because architecture changed (output layer)
            # This will load all matching layers (the encoder)
            model.load_state_dict(checkpoint['model_state_dict'], strict=False) 
            print("Model weights loaded (strict=False).")
            
            # Note: We don't load optimizer state as the head is different.
            # We will start from epoch 0 or the saved epoch, but optimizer is fresh.
            if 'epoch' in checkpoint:
                 start_epoch = checkpoint['epoch'] + 1
                 print(f"Resuming training from epoch {start_epoch + 1}.")
            else:
                 print("Starting fresh training from epoch 0.")

        except Exception as e:
            print(f"Error loading checkpoint: {e}. Starting fresh from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        total_correct = 0 # For accuracy
        total_samples = 0 # For accuracy

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            # label_batch is already a long tensor of class indices
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device) # (batch_size,)

            optimizer.zero_grad()
            out = model(pc_batch) # (batch_size, 16)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)
            
            # --- Calculate Accuracy ---
            _, predicted_class = torch.max(out, 1) # Get the index of the max logit
            total_correct += (predicted_class == label_batch).sum().item()
            total_samples += label_batch.size(0)

        # --- MODIFIED: Report Accuracy ---
        if total_samples > 0:
            avg_train_loss = total_train_loss / total_samples
            avg_train_acc = total_correct / total_samples
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:8.4f} | Train Accuracy: {avg_train_acc * 100:6.2f}%")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': avg_train_acc, # Save accuracy
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")

Physical Batch Size: 32
Filtered to 5 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting classification training with LEARNING_RATE = 0.001 for 20 epochs.
Scanning 5 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 5/5 [00:00<00:00, 18.61it/s]
/tmp/ipykernel_87127/4290524856.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load


Loaded a total of 40000 file pairs from all 'random_subset' files.
Total dataset size: 40000 files.
Model Initialized: PointNetPathLoss (Classifier)
Target Encoder Output Dimension: 512
Total Trainable Parameters: 845,008
Loading existing checkpoint from pointnet_classifier_model_800.pth
Model weights loaded (strict=False).
Resuming training from epoch 11.

--- Starting Training ---


Epoch 11/20 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 21.01it/s]


Epoch 11 | Train Loss:   1.3646 | Train Accuracy:  43.99%
Saving model checkpoint after epoch 11...
Model saved to pointnet_classifier_model_800.pth


...

Epoch 18 | Train Loss:   1.3199 | Train Accuracy:  45.29%
Saving model checkpoint after epoch 18...
Model saved to pointnet_classifier_model_800.pth


Epoch 19/20 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 21.04it/s]


Epoch 19 | Train Loss:   1.3131 | Train Accuracy:  45.78%
Saving model checkpoint after epoch 19...
Model saved to pointnet_classifier_model_800.pth


Epoch 20/20 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 21.02it/s]

Epoch 20 | Train Loss:   1.3134 | Train Accuracy:  45.55%
Saving model checkpoint after epoch 20...
Model saved to pointnet_classifier_model_800.pth
--- Training Complete ---


In [57]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- MODIFIED: Checkpoint, Feature Dim, and Class Config ---
FEATURE_DIM = 512 # REDUCED from 1536 to lower parameter count
NUM_CLASSES = 31  # (200 - 50) / 5 + 1 = 31 classes (50, 55, ..., 200)
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_800MHz.pth" # New path for new task

LEARNING_RATE = 0.001
NUM_EPOCHS = 30
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. MODIFIED: Dataset handles classification ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index."""
    # 1. Round to nearest 5
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    # 2. Clip to the valid range [50, 200]
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    # 3. Convert to class index: (50 -> 0, 55 -> 1, ..., 200 -> 30)
    class_index = (pl_clipped - 50.0) / 5.0
    # 4. Return as a long integer for CrossEntropyLoss
    return class_index.long()

class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() 
                    if not pair_id: 
                        continue
                        
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        print(f"\n  Warning: Files for '{pair_id}' listed but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32) # Load as float
        
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor_float = torch.from_numpy(label).squeeze()
        
        # --- MODIFIED: Convert float label to class index ---
        label_class_tensor = pl_to_class(label_tensor_float)
        
        return pc_tensor, label_class_tensor # Return class index

# --- 3. MODIFIED: Model head reduced and outputting 31 classes ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> (batch, 512, 7000)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :] 
        feat_stream = x[:, 3:, :] 
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1) 
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) 
        x = torch.max(x, 2)[0]  # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # --- MODIFIED: Final layer outputs NUM_CLASSES (31) ---
        self.fc1 = nn.Linear(feature_dim_encoder, 512) # 512 -> 512
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # Output 31 classes

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        x = self.encoder(x)    # -> (batch, 512)

        # Classification head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Do not squeeze

# --- 4. Main Training Script (Modified for Classification) ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    print(f"Starting classification training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
    )
    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty. Check paths and 'random_subset' files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # Model head is now for classification

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Classifier)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Target Output Classes: {NUM_CLASSES}")
    print(f"Total Trainable Parameters: {total_params:,}") # ~840k
    
    # --- MODIFIED: Loss and Optimizer ---
    criterion = torch.nn.CrossEntropyLoss() # Switched to CrossEntropy for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # ------------------------------------

    # --- Load Checkpoint ---
    start_epoch = 0
    # We check for the new model path
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Try to load the model weights
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully (strict=True).")
            
            # Try to load optimizer and epoch
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                 start_epoch = checkpoint['epoch'] + 1
                 print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
            else:
                 print("Optimizer/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            # This will happen if the old (16-class) model is at this path
            print(f"Error loading checkpoint (likely architecture mismatch): {e}.")
            print("Starting fresh training from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        total_correct = 0 # For accuracy
        total_samples = 0 # For accuracy
        avg_train_acc = 0.0 # Initialize for checkpoint saving

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device) # (batch_size,)

            optimizer.zero_grad()
            out = model(pc_batch) # (batch_size, 31)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)
            
            # --- Calculate Accuracy ---
            _, predicted_class = torch.max(out, 1) # Get the index of the max logit
            total_correct += (predicted_class == label_batch).sum().item()
            total_samples += label_batch.size(0)

        # --- MODIFIED: Report Accuracy ---
        if total_samples > 0:
            avg_train_loss = total_train_loss / total_samples
            avg_train_acc = total_correct / total_samples
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:8.4f} | Train Accuracy: {avg_train_acc * 100:6.2f}%")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': avg_train_acc, # Save accuracy
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")

Physical Batch Size: 32
Filtered to 5 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting classification training with LEARNING_RATE = 0.001 for 30 epochs.
Scanning 5 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 5/5 [00:00<00:00, 18.28it/s]
/tmp/ipykernel_87127/1653669814.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load


Loaded a total of 40000 file pairs from all 'random_subset' files.
Total dataset size: 40000 files.
Model Initialized: PointNetPathLoss (Classifier)
Target Encoder Output Dimension: 512
Target Output Classes: 31
Total Trainable Parameters: 846,943
Loading existing checkpoint from pointnet_classifier_model_31class_800MHz.pth
Model weights loaded successfully (strict=True).
Optimizer state loaded. Resuming training from epoch 21.

--- Starting Training ---


Epoch 21/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.69it/s]


Epoch 21 | Train Loss:   1.9863 | Train Accuracy:  27.77%
Saving model checkpoint after epoch 21...
Model saved to pointnet_classifier_model_31class_800MHz.pth


...

Epoch 28 | Train Loss:   1.9543 | Train Accuracy:  28.51%
Saving model checkpoint after epoch 28...
Model saved to pointnet_classifier_model_31class_800MHz.pth


Epoch 29/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.68it/s]


Epoch 29 | Train Loss:   1.9455 | Train Accuracy:  28.81%
Saving model checkpoint after epoch 29...
Model saved to pointnet_classifier_model_31class_800MHz.pth


Epoch 30/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.72it/s]

Epoch 30 | Train Loss:   1.9440 | Train Accuracy:  28.68%
Saving model checkpoint after epoch 30...
Model saved to pointnet_classifier_model_31class_800MHz.pth
--- Training Complete ---


In [59]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- MODIFIED: Checkpoint, Feature Dim, and Class Config ---
FEATURE_DIM = 512 # REDUCED from 1536 to lower parameter count
NUM_CLASSES = 31  # (200 - 50) / 5 + 1 = 31 classes (50, 55, ..., 200)
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_7GHz.pth" # New path for new task

LEARNING_RATE = 0.001
NUM_EPOCHS = 30
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7GHz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. MODIFIED: Dataset handles classification ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index."""
    # 1. Round to nearest 5
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    # 2. Clip to the valid range [50, 200]
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    # 3. Convert to class index: (50 -> 0, 55 -> 1, ..., 200 -> 30)
    class_index = (pl_clipped - 50.0) / 5.0
    # 4. Return as a long integer for CrossEntropyLoss
    return class_index.long()

class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() 
                    if not pair_id: 
                        continue
                        
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        print(f"\n  Warning: Files for '{pair_id}' listed but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32) # Load as float
        
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor_float = torch.from_numpy(label).squeeze()
        
        # --- MODIFIED: Convert float label to class index ---
        label_class_tensor = pl_to_class(label_tensor_float)
        
        return pc_tensor, label_class_tensor # Return class index

# --- 3. MODIFIED: Model head reduced and outputting 31 classes ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> (batch, 512, 7000)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :] 
        feat_stream = x[:, 3:, :] 
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1) 
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) 
        x = torch.max(x, 2)[0]  # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # --- MODIFIED: Final layer outputs NUM_CLASSES (31) ---
        self.fc1 = nn.Linear(feature_dim_encoder, 512) # 512 -> 512
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # Output 31 classes

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        x = self.encoder(x)    # -> (batch, 512)

        # Classification head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Do not squeeze

# --- 4. Main Training Script (Modified for Classification) ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    print(f"Starting classification training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
    )
    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty. Check paths and 'random_subset' files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # Model head is now for classification

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Classifier)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Target Output Classes: {NUM_CLASSES}")
    print(f"Total Trainable Parameters: {total_params:,}") # ~840k
    
    # --- MODIFIED: Loss and Optimizer ---
    criterion = torch.nn.CrossEntropyLoss() # Switched to CrossEntropy for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # ------------------------------------

    # --- Load Checkpoint ---
    start_epoch = 0
    # We check for the new model path
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Try to load the model weights
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully (strict=True).")
            
            # Try to load optimizer and epoch
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                 start_epoch = checkpoint['epoch'] + 1
                 print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
            else:
                 print("Optimizer/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            # This will happen if the old (16-class) model is at this path
            print(f"Error loading checkpoint (likely architecture mismatch): {e}.")
            print("Starting fresh training from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        total_correct = 0 # For accuracy
        total_samples = 0 # For accuracy
        avg_train_acc = 0.0 # Initialize for checkpoint saving

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device) # (batch_size,)

            optimizer.zero_grad()
            out = model(pc_batch) # (batch_size, 31)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)
            
            # --- Calculate Accuracy ---
            _, predicted_class = torch.max(out, 1) # Get the index of the max logit
            total_correct += (predicted_class == label_batch).sum().item()
            total_samples += label_batch.size(0)

        # --- MODIFIED: Report Accuracy ---
        if total_samples > 0:
            avg_train_loss = total_train_loss / total_samples
            avg_train_acc = total_correct / total_samples
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:8.4f} | Train Accuracy: {avg_train_acc * 100:6.2f}%")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': avg_train_acc, # Save accuracy
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")

Physical Batch Size: 32
Filtered to 5 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting classification training with LEARNING_RATE = 0.001 for 30 epochs.
Scanning 5 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 5/5 [00:00<00:00, 19.30it/s]
/tmp/ipykernel_87127/2304921788.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load


Loaded a total of 40000 file pairs from all 'random_subset' files.
Total dataset size: 40000 files.
Model Initialized: PointNetPathLoss (Classifier)
Target Encoder Output Dimension: 512
Target Output Classes: 31
Total Trainable Parameters: 846,943
Loading existing checkpoint from pointnet_classifier_model_31class_7GHz.pth
Model weights loaded successfully (strict=True).
Optimizer state loaded. Resuming training from epoch 11.

--- Starting Training ---


Epoch 11/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.71it/s]


Epoch 11 | Train Loss:   2.0885 | Train Accuracy:  28.67%
Saving model checkpoint after epoch 11...
Model saved to pointnet_classifier_model_31class_7GHz.pth


...

Epoch 28 | Train Loss:   1.9855 | Train Accuracy:  31.35%
Saving model checkpoint after epoch 28...
Model saved to pointnet_classifier_model_31class_7GHz.pth


Epoch 29/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.73it/s]


Epoch 29 | Train Loss:   1.9852 | Train Accuracy:  31.47%
Saving model checkpoint after epoch 29...
Model saved to pointnet_classifier_model_31class_7GHz.pth


Epoch 30/30 [Train]: 100%|██████████| 1250/1250 [01:00<00:00, 20.74it/s]

Epoch 30 | Train Loss:   1.9856 | Train Accuracy:  31.40%
Saving model checkpoint after epoch 30...
Model saved to pointnet_classifier_model_31class_7GHz.pth
--- Training Complete ---


In [60]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- MODIFIED: Checkpoint, Feature Dim, and Class Config ---
FEATURE_DIM = 512 # REDUCED from 1536 to lower parameter count
NUM_CLASSES = 31  # (200 - 50) / 5 + 1 = 31 classes (50, 55, ..., 200)
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_28GHz.pth" # New path for new task

LEARNING_RATE = 0.01
NUM_EPOCHS = 10
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. MODIFIED: Dataset handles classification ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index."""
    # 1. Round to nearest 5
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    # 2. Clip to the valid range [50, 200]
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    # 3. Convert to class index: (50 -> 0, 55 -> 1, ..., 200 -> 30)
    class_index = (pl_clipped - 50.0) / 5.0
    # 4. Return as a long integer for CrossEntropyLoss
    return class_index.long()

class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() 
                    if not pair_id: 
                        continue
                        
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        print(f"\n  Warning: Files for '{pair_id}' listed but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32) # Load as float
        
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor_float = torch.from_numpy(label).squeeze()
        
        # --- MODIFIED: Convert float label to class index ---
        label_class_tensor = pl_to_class(label_tensor_float)
        
        return pc_tensor, label_class_tensor # Return class index

# --- 3. MODIFIED: Model head reduced and outputting 31 classes ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> (batch, 512, 7000)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :] 
        feat_stream = x[:, 3:, :] 
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1) 
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) 
        x = torch.max(x, 2)[0]  # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # --- MODIFIED: Final layer outputs NUM_CLASSES (31) ---
        self.fc1 = nn.Linear(feature_dim_encoder, 512) # 512 -> 512
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # Output 31 classes

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        x = self.encoder(x)    # -> (batch, 512)

        # Classification head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Do not squeeze

# --- 4. Main Training Script (Modified for Classification) ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    print(f"Starting classification training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
    )
    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty. Check paths and 'random_subset' files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # Model head is now for classification

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Classifier)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Target Output Classes: {NUM_CLASSES}")
    print(f"Total Trainable Parameters: {total_params:,}") # ~840k
    
    # --- MODIFIED: Loss and Optimizer ---
    criterion = torch.nn.CrossEntropyLoss() # Switched to CrossEntropy for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # ------------------------------------

    # --- Load Checkpoint ---
    start_epoch = 0
    # We check for the new model path
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Try to load the model weights
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully (strict=True).")
            
            # Try to load optimizer and epoch
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                 start_epoch = checkpoint['epoch'] + 1
                 print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
            else:
                 print("Optimizer/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            # This will happen if the old (16-class) model is at this path
            print(f"Error loading checkpoint (likely architecture mismatch): {e}.")
            print("Starting fresh training from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        total_correct = 0 # For accuracy
        total_samples = 0 # For accuracy
        avg_train_acc = 0.0 # Initialize for checkpoint saving

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device) # (batch_size,)

            optimizer.zero_grad()
            out = model(pc_batch) # (batch_size, 31)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)
            
            # --- Calculate Accuracy ---
            _, predicted_class = torch.max(out, 1) # Get the index of the max logit
            total_correct += (predicted_class == label_batch).sum().item()
            total_samples += label_batch.size(0)

        # --- MODIFIED: Report Accuracy ---
        if total_samples > 0:
            avg_train_loss = total_train_loss / total_samples
            avg_train_acc = total_correct / total_samples
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:8.4f} | Train Accuracy: {avg_train_acc * 100:6.2f}%")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': avg_train_acc, # Save accuracy
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")

Physical Batch Size: 32
Filtered to 5 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting classification training with LEARNING_RATE = 0.01 for 10 epochs.
Scanning 5 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 5/5 [00:00<00:00,  5.75it/s]



Loaded a total of 40000 file pairs from all 'random_subset' files.
Total dataset size: 40000 files.
Model Initialized: PointNetPathLoss (Classifier)
Target Encoder Output Dimension: 512
Target Output Classes: 31
Total Trainable Parameters: 846,943
No existing model found at pointnet_classifier_model_31class_28GHz.pth. Starting fresh from epoch 0.

--- Starting Training ---


Epoch 1/10 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 20.95it/s]


Epoch 01 | Train Loss:   2.5711 | Train Accuracy:  17.26%
Saving model checkpoint after epoch 1...
Model saved to pointnet_classifier_model_31class_28GHz.pth


...

Epoch 08 | Train Loss:   2.2854 | Train Accuracy:  24.21%
Saving model checkpoint after epoch 8...
Model saved to pointnet_classifier_model_31class_28GHz.pth


Epoch 9/10 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 21.02it/s]


Epoch 09 | Train Loss:   2.2696 | Train Accuracy:  24.71%
Saving model checkpoint after epoch 9...
Model saved to pointnet_classifier_model_31class_28GHz.pth


Epoch 10/10 [Train]: 100%|██████████| 1250/1250 [00:59<00:00, 21.05it/s]

Epoch 10 | Train Loss:   2.2594 | Train Accuracy:  24.82%
Saving model checkpoint after epoch 10...
Model saved to pointnet_classifier_model_31class_28GHz.pth
--- Training Complete ---


In [61]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"

# --- MODIFIED: Checkpoint, Feature Dim, and Class Config ---
FEATURE_DIM = 512 # REDUCED from 1536 to lower parameter count
NUM_CLASSES = 31  # (200 - 50) / 5 + 1 = 31 classes (50, 55, ..., 200)
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_28GHz.pth" # New path for new task

LEARNING_RATE = 0.001
NUM_EPOCHS = 30
BATCH_SIZE = 32
NUM_WORKERS = 4
print(f"Physical Batch Size: {BATCH_SIZE}")
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]

# List of ALL potential folders (will be filtered)
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
ALL_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str
]
if not ALL_DATA_FOLDERS:
    raise ValueError("No data folders found matching the TX_SUBSET_SELECTED.")
print(f"Filtered to {len(ALL_DATA_FOLDERS)} folders for Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. MODIFIED: Dataset handles classification ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index."""
    # 1. Round to nearest 5
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    # 2. Clip to the valid range [50, 200]
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    # 3. Convert to class index: (50 -> 0, 55 -> 1, ..., 200 -> 30)
    class_index = (pl_clipped - 50.0) / 5.0
    # 4. Return as a long integer for CrossEntropyLoss
    return class_index.long()

class StandardPointCloudDataset(Dataset):
    def __init__(self, data_folders, base_dir):
        self.file_pairs = []
        print(f"Scanning {len(data_folders)} folders for 'random_subset' files...")
        
        total_files_loaded = 0

        for folder_name in tqdm(data_folders, desc="Scanning folders"):
            folder_path = os.path.join(base_dir, folder_name)
            subset_file_path = os.path.join(folder_path, "random_subset")

            if not os.path.exists(subset_file_path):
                print(f"\n  Warning: 'random_subset' file not found in '{folder_name}'. Skipping folder.")
                continue
            
            files_in_this_folder = 0
            try:
                with open(subset_file_path, 'r') as f:
                    pair_ids = f.readlines()
                
                for pair_id in pair_ids:
                    pair_id = pair_id.strip() 
                    if not pair_id: 
                        continue
                        
                    input_filename = f"{pair_id}_for_train.npy"
                    label_filename = f"{pair_id}_path_loss.npy"
                    
                    input_path = os.path.join(folder_path, input_filename)
                    label_path = os.path.join(folder_path, label_filename)

                    if os.path.exists(input_path) and os.path.exists(label_path):
                        self.file_pairs.append((input_path, label_path))
                        files_in_this_folder += 1
                    else:
                        print(f"\n  Warning: Files for '{pair_id}' listed but not found in '{folder_name}'. Skipping.")
                        
            except Exception as e:
                print(f"\n  Error reading 'random_subset' in '{folder_name}': {e}")
            
            total_files_loaded += files_in_this_folder

        print(f"\nLoaded a total of {total_files_loaded} file pairs from all 'random_subset' files.")
        print(f"Total dataset size: {len(self.file_pairs)} files.")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label = np.load(label_path).astype(np.float32) # Load as float
        
        pc_tensor = torch.from_numpy(pc_data)
        label_tensor_float = torch.from_numpy(label).squeeze()
        
        # --- MODIFIED: Convert float label to class index ---
        label_class_tensor = pl_to_class(label_tensor_float)
        
        return pc_tensor, label_class_tensor # Return class index

# --- 3. MODIFIED: Model head reduced and outputting 31 classes ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> (batch, 512, 7000)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :] 
        feat_stream = x[:, 3:, :] 
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1) 
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x))) 
        x = torch.max(x, 2)[0]  # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )

        # --- MODIFIED: Final layer outputs NUM_CLASSES (31) ---
        self.fc1 = nn.Linear(feature_dim_encoder, 512) # 512 -> 512
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # Output 31 classes

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)  # -> (batch, 7, 7000)
        x = self.encoder(x)    # -> (batch, 512)

        # Classification head
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Do not squeeze

# --- 4. Main Training Script (Modified for Classification) ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    print(f"Starting classification training with LEARNING_RATE = {LEARNING_RATE} for {NUM_EPOCHS} epochs.")

    # --- Create Datasets and Dataloaders ---
    train_dataset = StandardPointCloudDataset(
        data_folders=ALL_DATA_FOLDERS,
        base_dir=BASE_TRAIN_DIR
    )
    if len(train_dataset) == 0:
        raise ValueError("Dataset is empty. Check paths and 'random_subset' files.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False
    )

    # --- Initialize Model ---
    model = PointNetPathLoss().to(device) # Model head is now for classification

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model Initialized: {model.__class__.__name__} (Classifier)")
    print(f"Target Encoder Output Dimension: {FEATURE_DIM}")
    print(f"Target Output Classes: {NUM_CLASSES}")
    print(f"Total Trainable Parameters: {total_params:,}") # ~840k
    
    # --- MODIFIED: Loss and Optimizer ---
    criterion = torch.nn.CrossEntropyLoss() # Switched to CrossEntropy for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # ------------------------------------

    # --- Load Checkpoint ---
    start_epoch = 0
    # We check for the new model path
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"Loading existing checkpoint from {MODEL_SAVE_PATH}")
        try:
            checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
            
            # Try to load the model weights
            model.load_state_dict(checkpoint['model_state_dict'], strict=True) 
            print("Model weights loaded successfully (strict=True).")
            
            # Try to load optimizer and epoch
            if 'optimizer_state_dict' in checkpoint and 'epoch' in checkpoint:
                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                 start_epoch = checkpoint['epoch'] + 1
                 print(f"Optimizer state loaded. Resuming training from epoch {start_epoch + 1}.")
            else:
                 print("Optimizer/epoch not in checkpoint. Starting from epoch 0.")

        except Exception as e:
            # This will happen if the old (16-class) model is at this path
            print(f"Error loading checkpoint (likely architecture mismatch): {e}.")
            print("Starting fresh training from epoch 0.")
            start_epoch = 0
    else:
        print(f"No existing model found at {MODEL_SAVE_PATH}. Starting fresh from epoch 0.")
    # --------------------------------------------------

    print("\n--- Starting Training ---")

    for epoch in range(start_epoch, NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        total_correct = 0 # For accuracy
        total_samples = 0 # For accuracy
        avg_train_acc = 0.0 # Initialize for checkpoint saving

        for i, (pc_batch, label_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")):
            pc_batch = pc_batch.to(device)
            label_batch = label_batch.to(device) # (batch_size,)

            optimizer.zero_grad()
            out = model(pc_batch) # (batch_size, 31)
            loss = criterion(out, label_batch)

            if not torch.isfinite(loss):
                print(f"Warning: Encountered non-finite loss: {loss.item()} at step {i}. Skipping batch.")
                continue

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * pc_batch.size(0)
            
            # --- Calculate Accuracy ---
            _, predicted_class = torch.max(out, 1) # Get the index of the max logit
            total_correct += (predicted_class == label_batch).sum().item()
            total_samples += label_batch.size(0)

        # --- MODIFIED: Report Accuracy ---
        if total_samples > 0:
            avg_train_loss = total_train_loss / total_samples
            avg_train_acc = total_correct / total_samples
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:8.4f} | Train Accuracy: {avg_train_acc * 100:6.2f}%")
        else:
            print(f"Epoch {epoch+1:02d} | No data processed.")

        # --- Save Checkpoint After Each Epoch ---
        print(f"Saving model checkpoint after epoch {epoch+1}...")
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': avg_train_acc, # Save accuracy
            }, MODEL_SAVE_PATH)
            print(f"Model saved to {MODEL_SAVE_PATH}")
        except Exception as e:
            print(f"Error saving model checkpoint: {e}")
        # ----------------------------------------

    print("--- Training Complete ---")

Physical Batch Size: 32
Filtered to 5 folders for Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Starting classification training with LEARNING_RATE = 0.001 for 30 epochs.
Scanning 5 folders for 'random_subset' files...


Scanning folders: 100%|██████████| 5/5 [00:00<00:00, 21.07it/s]
/tmp/ipykernel_87127/2506507942.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load


Loaded a total of 40000 file pairs from all 'random_subset' files.
Total dataset size: 40000 files.
Model Initialized: PointNetPathLoss (Classifier)
Target Encoder Output Dimension: 512
Target Output Classes: 31
Total Trainable Parameters: 846,943
Loading existing checkpoint from pointnet_classifier_model_31class_28GHz.pth
Model weights loaded successfully (strict=True).
Optimizer state loaded. Resuming training from epoch 11.

--- Starting Training ---


Epoch 11/30 [Train]: 100%|██████████| 1250/1250 [02:18<00:00,  9.01it/s]


Epoch 11 | Train Loss:   2.2541 | Train Accuracy:  25.33%
Saving model checkpoint after epoch 11...
Model saved to pointnet_classifier_model_31class_28GHz.pth


...

Epoch 28 | Train Loss:   2.1491 | Train Accuracy:  27.77%
Saving model checkpoint after epoch 28...
Model saved to pointnet_classifier_model_31class_28GHz.pth


Epoch 29/30 [Train]: 100%|██████████| 1250/1250 [02:18<00:00,  9.01it/s]


Epoch 29 | Train Loss:   2.1400 | Train Accuracy:  28.19%
Saving model checkpoint after epoch 29...
Model saved to pointnet_classifier_model_31class_28GHz.pth


Epoch 30/30 [Train]: 100%|██████████| 1250/1250 [02:18<00:00,  9.00it/s]


Epoch 30 | Train Loss:   2.1403 | Train Accuracy:  28.13%
Saving model checkpoint after epoch 30...
Model saved to pointnet_classifier_model_31class_28GHz.pth
--- Training Complete ---


In [62]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_800MHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "800MHz" # Run for only one frequency
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 5 folders for evaluation for Freq: 800MHz, Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_800MHz.pth

--- Starting Evaluation ---

▶ Processing folder: 800MHz_processed__for_training_Tx_1


/tmp/ipykernel_87127/338536579.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|███████

  -> Folder RMSE (on rounded values):   9.0920 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_800MHz_Tx_1.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_18


  Evaluating: 100%|██████████| 826/826 [00:19<00:00, 42.57it/s]


  -> Folder RMSE (on rounded values):  11.5491 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_800MHz_Tx_18.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_19


...

  -> Folder RMSE (on rounded values):  10.8294 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_800MHz_Tx_19.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_2


  Evaluating: 100%|██████████| 727/727 [00:17<00:00, 42.46it/s]


  -> Folder RMSE (on rounded values):  10.4897 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_800MHz_Tx_2.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_8


  Evaluating: 100%|██████████| 751/751 [00:17<00:00, 42.57it/s]


  -> Folder RMSE (on rounded values):  11.5676 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_800MHz_Tx_8.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  10.6917 dB


In [ ]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_800MHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "800MHz" # Run for only one frequency
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

In [65]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_800MHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "800MHz" # Run for only one frequency
TX_SUBSET_SELECTED = [5,9,12,14,20]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_5", "28GHz_processed__for_training_Tx_9",
    "28GHz_processed__for_training_Tx_12", "28GHz_processed__for_training_Tx_14",
    "28GHz_processed__for_training_Tx_20",
    "7GHz_processed__for_training_Tx_5", "7GHz_processed__for_training_Tx_9",
    "7GHz_processed__for_training_Tx_12", "7GHz_processed__for_training_Tx_14",
    "7GHz_processed__for_training_Tx_20",
    "800MHz_processed__for_training_Tx_5", "800MHz_processed__for_training_Tx_9",
    "800MHz_processed__for_training_Tx_12", "800MHz_processed__for_training_Tx_14",
    "800MHz_processed__for_training_Tx_20"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 5 folders for evaluation for Freq: 800MHz, Tx IDs: [5, 9, 12, 14, 20]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_800MHz.pth

--- Starting Evaluation ---

▶ Processing folder: 800MHz_processed__for_training_Tx_5


/tmp/ipykernel_87127/1428384949.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|██████

  -> Folder RMSE (on rounded values):  17.8930 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_5.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_9


  Evaluating: 100%|██████████| 762/762 [00:13<00:00, 57.48it/s]


  -> Folder RMSE (on rounded values):  17.3058 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_9.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_12


...

  -> Folder RMSE (on rounded values):  23.7526 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_12.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_14


  Evaluating: 100%|██████████| 718/718 [00:12<00:00, 57.49it/s]


  -> Folder RMSE (on rounded values):  16.2086 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_14.csv

▶ Processing folder: 800MHz_processed__for_training_Tx_20


  Evaluating: 100%|██████████| 696/696 [00:12<00:00, 57.09it/s]

  -> Folder RMSE (on rounded values):  15.2857 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_800MHz_Tx_20.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  18.3914 dB


In [63]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_7GHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "7GHz" # Run for only one frequency
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 5 folders for evaluation for Freq: 7GHz, Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_7GHz.pth

--- Starting Evaluation ---

▶ Processing folder: 7GHz_processed__for_training_Tx_1


/tmp/ipykernel_87127/3447722956.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|██████

  -> Folder RMSE (on rounded values):  11.6939 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_7GHz_Tx_1.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_18


  Evaluating: 100%|██████████| 826/826 [00:19<00:00, 41.77it/s]


  -> Folder RMSE (on rounded values):  12.5777 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_7GHz_Tx_18.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_19


...

  -> Folder RMSE (on rounded values):  13.8717 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_7GHz_Tx_19.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_2


  Evaluating: 100%|██████████| 727/727 [00:17<00:00, 42.41it/s]


  -> Folder RMSE (on rounded values):  12.0951 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_7GHz_Tx_2.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_8


  Evaluating: 100%|██████████| 751/751 [00:17<00:00, 42.50it/s]


  -> Folder RMSE (on rounded values):  15.2090 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_7GHz_Tx_8.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  13.0704 dB


In [66]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_7GHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "7GHz" # Run for only one frequency
TX_SUBSET_SELECTED = [5,9,12,14,20]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_5", "28GHz_processed__for_training_Tx_9",
    "28GHz_processed__for_training_Tx_12", "28GHz_processed__for_training_Tx_14",
    "28GHz_processed__for_training_Tx_20",
    "7GHz_processed__for_training_Tx_5", "7GHz_processed__for_training_Tx_9",
    "7GHz_processed__for_training_Tx_12", "7GHz_processed__for_training_Tx_14",
    "7GHz_processed__for_training_Tx_20",
    "800MHz_processed__for_training_Tx_5", "800MHz_processed__for_training_Tx_9",
    "800MHz_processed__for_training_Tx_12", "800MHz_processed__for_training_Tx_14",
    "800MHz_processed__for_training_Tx_20"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 5 folders for evaluation for Freq: 7GHz, Tx IDs: [5, 9, 12, 14, 20]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_7GHz.pth

--- Starting Evaluation ---

▶ Processing folder: 7GHz_processed__for_training_Tx_5


/tmp/ipykernel_87127/1476253964.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|██████

  -> Folder RMSE (on rounded values):  19.7999 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_7GHz_Tx_5.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_9


  Evaluating: 100%|██████████| 762/762 [00:13<00:00, 58.47it/s]


  -> Folder RMSE (on rounded values):  19.8179 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_7GHz_Tx_9.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_12


...

  -> Folder RMSE (on rounded values):  25.7740 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_7GHz_Tx_12.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_14


  Evaluating: 100%|██████████| 718/718 [00:12<00:00, 58.19it/s]


  -> Folder RMSE (on rounded values):  18.8268 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_7GHz_Tx_14.csv

▶ Processing folder: 7GHz_processed__for_training_Tx_20


  Evaluating: 100%|██████████| 696/696 [00:12<00:00, 56.97it/s]


  -> Folder RMSE (on rounded values):  19.2944 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_7GHz_Tx_20.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  20.9038 dB


In [64]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/train"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_28GHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "28GHz" # Run for only one frequency
TX_SUBSET_SELECTED = [1, 18, 8, 19, 2]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_1", "28GHz_processed__for_training_Tx_10",
    "28GHz_processed__for_training_Tx_11", "28GHz_processed__for_training_Tx_13",
    "28GHz_processed__for_training_Tx_15", "28GHz_processed__for_training_Tx_16",
    "28GHz_processed__for_training_Tx_17", "28GHz_processed__for_training_Tx_18",
    "28GHz_processed__for_training_Tx_19", "28GHz_processed__for_training_Tx_2",
    "28GHz_processed__for_training_Tx_3", "28GHz_processed__for_training_Tx_4",
    "28GHz_processed__for_training_Tx_6", "28GHz_processed__for_training_Tx_7",
    "28GHz_processed__for_training_Tx_8",
    "7GHz_processed__for_training_Tx_1", "7GHz_processed__for_training_Tx_10",
    "7GHz_processed__for_training_Tx_11", "7GHz_processed__for_training_Tx_13",
    "7GHz_processed__for_training_Tx_15", "7GHz_processed__for_training_Tx_16",
    "7Gzz_processed__for_training_Tx_17", "7GHz_processed__for_training_Tx_18",
    "7GHz_processed__for_training_Tx_19", "7GHz_processed__for_training_Tx_2",
    "7GHz_processed__for_training_Tx_3", "7GHz_processed__for_training_Tx_4",
    "7GHz_processed__for_training_Tx_6", "7GHz_processed__for_training_Tx_7",
    "7GHz_processed__for_training_Tx_8",
    "800MHz_processed__for_training_Tx_1", "800MHz_processed__for_training_Tx_10",
    "800MHz_processed__for_training_Tx_11", "800MHz_processed__for_training_Tx_13",
    "800MHz_processed__for_training_Tx_15", "800MHz_processed__for_training_Tx_16",
    "800MHz_processed__for_training_Tx_17", "800MHz_processed__for_training_Tx_18",
    "800MHz_processed__for_training_Tx_19", "800MHz_processed__for_training_Tx_2",
    "800MHz_processed__for_training_Tx_3", "800MHz_processed__for_training_Tx_4",
    "800MHz_processed__for_training_Tx_6", "800MHz_processed__for_training_Tx_7",
    "800MHz_processed__for_training_Tx_8"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

/tmp/ipykernel_87127/2710928866.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)


Found 5 folders for evaluation for Freq: 28GHz, Tx IDs: [1, 18, 8, 19, 2]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_28GHz.pth

--- Starting Evaluation ---

▶ Processing folder: 28GHz_processed__for_training_Tx_1


  Evaluating: 100%|██████████| 922/922 [00:21<00:00, 42.58it/s]


  -> Folder RMSE (on rounded values):  14.0091 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_28GHz_Tx_1.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_18


  Evaluating: 100%|██████████| 824/824 [00:19<00:00, 42.57it/s]


...

  -> Folder RMSE (on rounded values):  15.5435 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_28GHz_Tx_19.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_2


  Evaluating: 100%|██████████| 721/721 [00:16<00:00, 42.71it/s]


  -> Folder RMSE (on rounded values):  14.1761 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_28GHz_Tx_2.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_8


  Evaluating: 100%|██████████| 732/732 [00:17<00:00, 42.45it/s]

  -> Folder RMSE (on rounded values):  16.0752 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/train/test_28GHz_Tx_8.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  14.8195 dB


In [67]:
import numpy as np
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re 

# --- 1. Configuration ---

BASE_TRAIN_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
TEST_SAVE_DIR = "/home/mkrishne/PL_competition/extracted_regions/test"
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

# --- MODIFIED: Model Configuration ---
MODEL_SAVE_PATH = "pointnet_classifier_model_31class_28GHz.pth" # Classifier model
FEATURE_DIM = 512    # Must match the trained classifier
NUM_CLASSES = 31     # Must match the trained classifier (50-200 in 5dB steps)

# --- Evaluation Run Configuration ---
FREQ_TO_TEST = "28GHz" # Run for only one frequency
TX_SUBSET_SELECTED = [5,9,12,14,20]
BATCH_SIZE = 32
NUM_WORKERS = 4

# List of ALL potential folders for the test set
ALL_DATA_FOLDERS_POTENTIAL = [
    "28GHz_processed__for_training_Tx_5", "28GHz_processed__for_training_Tx_9",
    "28GHz_processed__for_training_Tx_12", "28GHz_processed__for_training_Tx_14",
    "28GHz_processed__for_training_Tx_20",
    "7GHz_processed__for_training_Tx_5", "7GHz_processed__for_training_Tx_9",
    "7GHz_processed__for_training_Tx_12", "7GHz_processed__for_training_Tx_14",
    "7GHz_processed__for_training_Tx_20",
    "800MHz_processed__for_training_Tx_5", "800MHz_processed__for_training_Tx_9",
    "800MHz_processed__for_training_Tx_12", "800MHz_processed__for_training_Tx_14",
    "800MHz_processed__for_training_Tx_20"
]

# --- Filter folders based on TX_SUBSET_SELECTED AND FREQ_TO_TEST ---
tx_subset_str = {str(tx_id) for tx_id in TX_SUBSET_SELECTED}
TEST_DATA_FOLDERS = [
    folder for folder in ALL_DATA_FOLDERS_POTENTIAL
    if folder.split('_')[-1] in tx_subset_str and folder.startswith(FREQ_TO_TEST)
]
if not TEST_DATA_FOLDERS:
    raise ValueError(f"No folders found for Tx IDs {TX_SUBSET_SELECTED} and Freq {FREQ_TO_TEST}.")
print(f"Found {len(TEST_DATA_FOLDERS)} folders for evaluation for Freq: {FREQ_TO_TEST}, Tx IDs: {TX_SUBSET_SELECTED}")
# -------------------------------------------------

# --- 2. Label Conversion Functions ---

def pl_to_class(pl_tensor):
    """Converts a raw path loss value to a class index (0-30)."""
    pl_rounded = torch.round(pl_tensor / 5.0) * 5.0
    pl_clipped = torch.clamp(pl_rounded, 50.0, 200.0)
    class_index = (pl_clipped - 50.0) / 5.0
    return class_index.long()

def class_to_pl(class_index_tensor):
    """Converts a class index (0-30) back to a path loss value (50-200)."""
    # Note: Needs to be done with torch tensors to keep on-device
    return (class_index_tensor.float() * 5.0) + 50.0

# --- 3. Dataset Definition for Evaluation ---

class EvaluationPointCloudDataset(Dataset):
    def __init__(self, folder_path):
        self.file_pairs = []
        input_files = glob.glob(os.path.join(folder_path, "*_for_train.npy"))
        
        for input_path in input_files:
            label_path = input_path.replace("_for_train.npy", "_path_loss.npy")
            if os.path.exists(label_path):
                self.file_pairs.append((input_path, label_path))
            else:
                print(f"Warning: Missing label for {input_path}. Skipping file.")
        if not self.file_pairs:
            print(f"Warning: No valid file pairs found in {folder_path}")

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        input_path, label_path = self.file_pairs[idx]
        pc_data = np.load(input_path).astype(np.float32)
        label_original_float = np.load(label_path).astype(np.float32)
        
        pc_tensor = torch.from_numpy(pc_data)
        label_original_tensor = torch.from_numpy(label_original_float).squeeze()
        
        # Also return the class index for calculating rounded truth
        label_class_tensor = pl_to_class(label_original_tensor)
        
        return pc_tensor, label_class_tensor, label_original_tensor

# --- 4. Model Definitions (CLASSIFIER VERSION) ---

class PointNetEncoder(nn.Module):
    def __init__(self, pos_dim=3, feat_dim=4, feature_dim_out=FEATURE_DIM):
        super().__init__()
        self.conv1_pos = nn.Conv1d(pos_dim, 64, 1)
        self.conv2_pos = nn.Conv1d(64, 128, 1)
        self.bn1_pos = nn.BatchNorm1d(64)
        self.bn2_pos = nn.BatchNorm1d(128)
        self.conv1_feat = nn.Conv1d(feat_dim, 64, 1)
        self.conv2_feat = nn.Conv1d(64, 128, 1)
        self.bn1_feat = nn.BatchNorm1d(64)
        self.bn2_feat = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, feature_dim_out, 1) # -> 512
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(feature_dim_out)

    def forward(self, x):
        pos_stream = x[:, :3, :]
        feat_stream = x[:, 3:, :]
        pos_out = F.relu(self.bn1_pos(self.conv1_pos(pos_stream)))
        pos_out = F.relu(self.bn2_pos(self.conv2_pos(pos_out)))
        feat_out = F.relu(self.bn1_feat(self.conv1_feat(feat_stream)))
        feat_out = F.relu(self.bn2_feat(self.conv2_feat(feat_out)))
        combined = torch.cat([pos_out, feat_out], dim=1)
        x = F.relu(self.bn3(self.conv3(combined)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.max(x, 2)[0] # -> (batch, 512)
        return x

class PointNetPathLoss(nn.Module):
    def __init__(self, input_dim_pos=3, input_dim_feat=4, feature_dim_encoder=FEATURE_DIM):
        super().__init__()
        self.encoder = PointNetEncoder(
            pos_dim=input_dim_pos, 
            feat_dim=input_dim_feat, 
            feature_dim_out=feature_dim_encoder
        )
        self.fc1 = nn.Linear(feature_dim_encoder, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, NUM_CLASSES) # -> 31 outputs
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x) # -> (batch, 31)
        return x # Return logits

# --- 5. Main Evaluation Script ---

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = PointNetPathLoss().to(device)
    print("Model initialized (Classifier).")

    if not os.path.exists(MODEL_SAVE_PATH):
        raise FileNotFoundError(f"Model checkpoint not found at {MODEL_SAVE_PATH}.")
    
    try:
        checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        print(f"Successfully loaded model weights from {MODEL_SAVE_PATH}")
    except Exception as e:
        raise RuntimeError(f"Error loading checkpoint: {e}. Ensure model definition matches checkpoint.")

    model.eval()

    all_predictions_rounded = []
    all_labels_rounded = []
    all_labels_original = [] # For the CSV

    print("\n--- Starting Evaluation ---")
    
    for folder_name in TEST_DATA_FOLDERS:
        print(f"\n▶ Processing folder: {folder_name}")
        folder_path = os.path.join(BASE_TRAIN_DIR, folder_name)
        
        test_dataset = EvaluationPointCloudDataset(folder_path=folder_path)
        if len(test_dataset) == 0:
            print("  -> No files found. Skipping folder.")
            continue
            
        test_loader = DataLoader(
            test_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=(device.type == 'cuda')
        )
        
        # Lists for this folder's results
        folder_preds_list = []
        folder_true_rounded_list = []
        folder_true_original_list = []

        with torch.no_grad():
            for pc_batch, label_class_batch, label_original_batch in tqdm(test_loader, desc="  Evaluating"):
                pc_batch = pc_batch.to(device)
                label_class_batch = label_class_batch.to(device) # True class index
                # label_original_batch stays on CPU, we just need it for the CSV
                
                out_logits = model(pc_batch) # -> (batch, 31)
                
                # Get predicted class index
                _, predicted_class_idx = torch.max(out_logits, 1)
                
                # --- Convert indices back to dB values for RMSE and CSV ---
                predicted_pl = class_to_pl(predicted_class_idx)
                true_rounded_pl = class_to_pl(label_class_batch)
                
                # Store results
                folder_preds_list.append(predicted_pl.cpu().numpy())
                folder_true_rounded_list.append(true_rounded_pl.cpu().numpy())
                folder_true_original_list.append(label_original_batch.cpu().numpy())
        
        # Concatenate all batch results
        predictions = np.concatenate(folder_preds_list)
        labels_rounded = np.concatenate(folder_true_rounded_list)
        labels_original = np.concatenate(folder_true_original_list)
        
        all_predictions_rounded.append(predictions)
        all_labels_rounded.append(labels_rounded)
        
        # Calculate folder-specific RMSE using ROUNDED values
        folder_mse = np.mean((predictions - labels_rounded)**2)
        folder_rmse = np.sqrt(folder_mse)
        print(f"  -> Folder RMSE (on rounded values): {folder_rmse:8.4f} dB")
        
        # --- Create and Save CSV ---
        match = re.match(r"(\w+)_processed__for_training_Tx_(\d+)", folder_name)
        if not match:
            print(f"  -> ERROR: Could not parse folder name '{folder_name}'. Skipping CSV save.")
            continue
            
        freq = match.group(1)
        tx_id = match.group(2)
        
        csv_filename = f"test_{freq}_Tx_{tx_id}.csv"
        csv_save_path = os.path.join(TEST_SAVE_DIR, csv_filename)
        
        # Create DataFrame with the 3 requested columns
        results_df = pd.DataFrame({
            'predicted_pathloss': predictions,
            'true_pathloss_rounded': labels_rounded,
            'original_pathloss': labels_original
        })
        
        results_df.to_csv(csv_save_path, index=False)
        print(f"  -> Results saved to: {csv_save_path}")

    # --- Calculate and Print Overall RMSE ---
    if all_predictions_rounded:
        overall_preds = np.concatenate(all_predictions_rounded)
        overall_labels = np.concatenate(all_labels_rounded)
        
        overall_mse = np.mean((overall_preds - overall_labels)**2)
        overall_rmse = np.sqrt(overall_mse)
        
        print("\n--- Evaluation Complete ---")
        print(f"Overall Test RMSE (on rounded values): {overall_rmse:8.4f} dB")
    else:
        print("\n--- Evaluation Complete ---")
        print("No test data was processed.")

Found 5 folders for evaluation for Freq: 28GHz, Tx IDs: [5, 9, 12, 14, 20]
Using device: cuda
Model initialized (Classifier).
Successfully loaded model weights from pointnet_classifier_model_31class_28GHz.pth

--- Starting Evaluation ---

▶ Processing folder: 28GHz_processed__for_training_Tx_5


/tmp/ipykernel_87127/2420061968.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
  Evaluating: 100%|██████

  -> Folder RMSE (on rounded values):  22.8715 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_5.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_9


  Evaluating: 100%|██████████| 734/734 [00:12<00:00, 56.78it/s]


  -> Folder RMSE (on rounded values):  20.7851 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_9.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_12


...

  -> Folder RMSE (on rounded values):  28.6119 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_12.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_14


  Evaluating: 100%|██████████| 701/701 [00:12<00:00, 57.96it/s]


  -> Folder RMSE (on rounded values):  18.3972 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_14.csv

▶ Processing folder: 28GHz_processed__for_training_Tx_20


  Evaluating: 100%|██████████| 685/685 [00:11<00:00, 57.34it/s]

  -> Folder RMSE (on rounded values):  20.0386 dB
  -> Results saved to: /home/mkrishne/PL_competition/extracted_regions/test/test_28GHz_Tx_20.csv

--- Evaluation Complete ---
Overall Test RMSE (on rounded values):  22.4732 dB
